In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from matplotlib import pyplot as plt
from matplotlib import image as IMG
import imageio
import PIL
from PIL import Image, ImageEnhance
import torchvision as tv
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from skimage.transform import rescale, resize, downscale_local_mean
from pathlib import Path
import torchvision.transforms #import (Compose, ColorJitter, RandomGrayscale, RandomHorizontalFlip, RandomOrder, RandomRotation)

In [ ]:
if torch.cuda.is_available():     # Make sure GPU is available
    dev = torch.device("cuda:0")
    kwar = {'num_workers': 8, 'pin_memory': True}
    cpu = torch.device("cpu")
else:
    print("Warning: CUDA not found, CPU only.")
    dev = torch.device("cpu")
    kwar = {}
    cpu = torch.device("cpu")

np.random.seed(551)

In [ ]:
images_path = 'chexpertData_new/'
images_path_new = 'GB7_new/'
labels_path = 'train_new.csv'
test_path = 'test_2.csv'

In [ ]:
train_df = pd.read_csv(labels_path)
#train_df

In [ ]:
frontal = list(map(str, train_df[train_df['label']=='frontal']['target_name'])) 
lateral = list(map(str, train_df[train_df['label']=='lateral']['target_name']))

In [ ]:
images = np.concatenate([frontal, lateral])
image_names = []
image_labels = {}

for image in images:
    image_name = images_path + '/' + image
    if image in frontal:
        image_labels.update({image_name: 1})
    else:
        image_labels.update({image_name: 0})
    image_names.append(image_name)

In [ ]:
from tqdm import tqdm

import cv2
resized_images = {}
for image_name in tqdm(image_names):
  image = cv2.imread(image_name, 0)
  resized_image = cv2.resize(image, (64, 64))
  resized_images[image_name] = resized_image

In [ ]:
for i, k in enumerate(resized_images):
    if i == 0:
        print(resized_images[k].shape)
        plt.imshow(resized_images[k], cmap='gray')
        break

In [ ]:
class LungsClassificationDataset(Dataset):
    def __init__(self, image_names, resized_images, image_labels, transformations=None):
        self.image_names = image_names
        self.resized_images = resized_images
        self.image_labels = image_labels
        self.transformations = transformations
                                            
    def __len__(self):
        return len(self.image_names)
        
    def __getitem__(self, index):
        image_path = self.image_names[index]

        #image_0 = self.resized_images[image_path]
        image = Image.open(image_path)
        label = self.image_labels[image_path]

        if self.transformations:
            image = self.transformations(image)
        
        image = (np.asarray(image.convert("L")) / 256) ** 0.4

        image = torch.tensor(image)
        label = torch.tensor(label)

        result = {"image":image,"label":label}
        return result

In [ ]:
train_image_names, val_image_names = train_test_split(image_names, test_size=0.2)
transform = torchvision.transforms.Compose([torchvision.transforms.ColorJitter(), torchvision.transforms.RandomHorizontalFlip(p=0.5)])

train = LungsClassificationDataset(train_image_names, resized_images, image_labels, transformations=transform)
validation = LungsClassificationDataset(val_image_names, resized_images, image_labels, transformations=transform)

In [ ]:
imageWidth, imageHeight = train[0]['image'].shape

In [ ]:
for i in tqdm(range((len(train)))):
    sample = train[i]
    plt.imshow(sample['image'], cmap='gray')
    plt.show()
    print(sample['label'])

In [ ]:
sample = train[22]
plt.imshow(sample['image'], cmap='gray')
print(sample['label'])

In [ ]:
len(validation)

In [ ]:
class LungModel(nn.Module):
    def __init__(self,width,height): 
        super(LungModel,self).__init__()  

        numConvs1 = 8
        numConvs2 = 16
        numConvs3 = 32
        numConvs4 = 64
        convSize = 3
       
        self.cnv1 = nn.Conv2d(in_channels=1, out_channels=numConvs1, kernel_size=convSize)
        self.cnv2 = nn.Conv2d(in_channels=numConvs1, out_channels=numConvs2, kernel_size=convSize)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        self.cnv3 = nn.Conv2d(in_channels=numConvs2, out_channels=numConvs3, kernel_size=convSize)
        self.cnv4 = nn.Conv2d(in_channels=numConvs3, out_channels=numConvs4, kernel_size=convSize)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        denseSize1 = 512
        denseSize2 = 100
      
        self.dense1 = nn.Linear(64*13*13, denseSize1)
        self.dense2 = nn.Linear(denseSize1, denseSize2)
        self.dense3 = nn.Linear(denseSize2, 1)
        self.dropout = nn.Dropout(0.25)
        
    def forward(self,x):

        x = F.elu(self.cnv1(x))
        x = F.elu(self.cnv2(x))
        x = self.maxpool1(x)
        x = F.elu(self.cnv3(x))
        x = F.elu(self.cnv4(x))
        x = self.maxpool2(x)
        x = x.view(-1,self.num_flat_features(x))
        x = F.elu(self.dense1(x)) 
        x = self.dropout(x)
        x = F.elu(self.dense2(x)) 
        x = self.dropout(x)
        x = self.dense3(x)        
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [ ]:
model = LungModel(imageWidth, imageHeight).to(dev)

In [ ]:
train_loader = DataLoader(train, batch_size=50, shuffle=True, drop_last=True)
val_loader = DataLoader(validation, batch_size=50, shuffle=True, drop_last=True)
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)

In [ ]:
import time
print(time.time())

In [ ]:
num_epochs = 20

t = time.time() 

for epoch in range(num_epochs):

    model.train()
    train_loss = 0
    time_start = time.time()

    for sample_idx, sample in enumerate(train_loader):
        image, label = sample['image'].unsqueeze(1).float(), sample['label'].unsqueeze(1).float()
        image, label = image.to(dev), label.to(dev)

        prediction = model(image)

        optimizer.zero_grad()
        loss = criterion(prediction, label)
        loss.backward()
        optimizer.step() 

        train_loss += loss.item()

        if sample_idx % 100 == 0:
            print(loss.item())

    val_loss = 0
    model.eval()

    with torch.no_grad():
        for sample in val_loader:
            image, label = sample['image'].unsqueeze(1).float(), sample['label'].unsqueeze(1).float()
            image, label = image.to(dev), label.to(dev)
            prediction = model(image)

            optimizer.zero_grad()
            loss = criterion(prediction, label)
            val_loss += loss.item()

    train_loss /= len(train_loader)
    val_loss /= len(val_loader)

    print(f"Epoch: {epoch}; Time Epoch: {time.time() - time_start}\nTraining Loss: {train_loss}\nValidation Loss: {val_loss}\n-----")